# HW11

Each of these questions is related to material in Topics 2.5 and 2.7.

In [2]:
name = "Leonid"              # Enter your name
user = "rempell"              # Enter your BC username
print("HW11 submission for {} ({})".format(name,user))

HW11 submission for Leonid (rempell)


Please rename the file so that it includes your BC username. For example, I would change <b>HW11</b> to <b>HW11-reuterj</b> and upload <b>HW11-reuterj.ipynb</b> to Canvas.

## Lots of useful code

In [3]:
import os
import numpy as np
import statsmodels.api as sm

In [4]:
# Formats how numbers are displayed by numpy
def float_formatter(x):
    return "{:7.4f}".format(x)
np.set_printoptions(formatter={'float_kind':float_formatter})

In [5]:
# Function that runs OLS regression
# Note: y and x should be structured arrays with data sorted in columns

def reg_m(y, x):
    X = sm.add_constant(x)    # adds column of ones to X
    results = sm.OLS(y, X).fit()                # creates object containing regression results
    return results

#reg_out = reg_m(y,X)
#reg_out.summary()
#reg_out.params
#reg_out.pvalues

In [6]:
# Imports daily return data from Fama and French
# Begins in 19260701 and ends in 20190930

f = open('Factors_Monthly.csv', 'r')
f_list = f.readlines()
f.close()

ff1 = [element.rstrip().split(',') for element in f_list][1:]
ff2 = sorted([[l[0], l[1], float(l[2])/100, float(l[3])/100, float(l[4])/100, float(l[5])/100] for l in ff1])

mktrf_yymm = np.array([tup[0] for tup in ff2])    # date strings
mktrf_yyyy = np.array([tup[1] for tup in ff2])    # date strings

mktrf_cols = np.array(["MKTRF"])
mktrf_ret  = np.array([tup[3] for tup in ff2])

print(mktrf_ret.shape, mktrf_yymm.shape, mktrf_cols.shape)
print(mktrf_ret[0], mktrf_ret[-1])
print(mktrf_yymm[0], mktrf_yymm[-1])

(1119,) (1119,) (1,)
0.0296 0.0144
192607 201909


<b>(3 points) Question 1.</b> Pick a consecutive 120 month period that occurs between January 1928 and September 2019 and pick a single calendar month. Use monthly return data for 'mrktf_ret' (created above) to test whether the excess return on the market is statistically significantly different during this calendar month than during the other eleven months. Run the appropriate regression, print your chosen month, print the estimated coefficient on the indicator variable for your chosen month, and print the p-value from the hypothesis test that this coefficient is equal to zero (in a two-sided test, which is the default assumption when reporting p-values).

In [7]:
# I recommend using list comprehension to create variable that equals one during chosen month and zero otherwise
# This is a trick for converting an array back into a list
listdate = mktrf_yymm.tolist()
print(listdate[0], listdate[0][-2:], int(listdate[0][-2:]))

192607 07 7


In [8]:
r = mktrf_ret.tolist()
start = 0
for i in range(len(listdate)):
    if listdate[i][0:4] == '1998':
        start = i
        
g = r[start:start+120]

june = [0 for element in range(1,121,1)]

x = 6
june[x] = 1
while x <= len(june):
    x = x+12
    if x <= len(june):
        june[x] = 1
    
mr = np.array(g)
j = np.array(june)

reg_out = reg_m(np.reshape(mr,(120,1)),np.reshape(j,(120,1)))

print('The chosen month was June and the period is the 120 month period following January 1998\n')
print(reg_out.summary())
print('\nThe estimated coeffcient of June on returns during this preiod is: {}'.format(float_formatter(reg_out.params[1])))
print('The p value of June on returns during this preiod is: {}'.format(float_formatter(reg_out.pvalues[1])))

The chosen month was June and the period is the 120 month period following January 1998

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.1212
Date:                Tue, 17 Dec 2019   Prob (F-statistic):              0.728
Time:                        00:08:44   Log-Likelihood:                 200.42
No. Observations:                 120   AIC:                            -396.8
Df Residuals:                     118   BIC:                            -391.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------

<b>(3 points) Question 2.</b> Pick one of your two stocks from Project 3 Part 1. Use all non-missing daily returns for this stock between October 2015 and September 2019 to estimate the Fama-French three-factor model. Recall that the dependent variable should be the daily stock return in excess of the daily risk-free rate of return.

Print a table with three columns. Column one will contain “alpha” “beta on MKTRF”, “beta on SMB”, and “beta on HML”, column two will contain the corresponding estimated coefficients, and column three will contain the p-values from the hypothesis tests that the estimated coefficients for “alpha” is equal to zero, …, from the hypothesis test that the estimated coefficients for “beta on HML” is equal to zero.

In [9]:
os.chdir('/Users/leonrem/Desktop/Data Analytics/Projects/Project 3')   # Where .npy are stored on my computer
ff_rets = np.load('Project3_ret.npy')
ff_cols = np.load('Project3_ret_cols.npy')
ff_date = np.load('Project3_ret_rows.npy')
costco = np.load('cost.npy')
dates = np.load('dates_cost.npy')

In [10]:
# Code that I use to test that stock return array is the same size as ff_date array
print(ff_date.shape==dates.shape)
print(ff_date[0]==dates[0])
print(ff_date[-1]==dates[-1])

True
True
True


In [11]:
print(ff_cols[0])
print(ff_rets)

MKTRF
[[ 0.0013 -0.0045 -0.0004 ...  0.0063 -0.0656 -0.0074]
 [ 0.0148  0.0034 -0.0084 ... -0.0011 -0.0169 -0.0071]
 [ 0.0193  0.0062  0.0077 ...  0.0130 -0.0736  0.0254]
 ...
 [-0.0041 -0.0096  0.0011 ... -0.0434 -0.0243 -0.0155]
 [-0.0062 -0.0035  0.0088 ...  0.0164  0.0479 -0.0072]
 [ 0.0050 -0.0014 -0.0049 ...  0.0051  0.0295 -0.0057]]


In [12]:
rf    = ff_rets[:,ff_cols=='RF']
dep_var = costco.reshape(1006,1) - rf
mktrf = ff_rets[:,ff_cols=='MKTRF']
smb = ff_rets[:,ff_cols=='SMB']
hml = ff_rets[:,ff_cols=='HML']

X = np.hstack((mktrf,smb,hml))

reg_out = reg_m(dep_var,X)

vars = ['alpha\t     ','beta on MKTRF','beta on SMB  ','beta on HML  ']
print("\t\t     Coef.   P-value")
print("")
for num in range(len(vars)):
    print("{}      {:7.4f}  {:7.4f}".
         format(vars[num],reg_out.params[num],reg_out.pvalues[num]))

		     Coef.   P-value

alpha	            0.0008   0.0333
beta on MKTRF      -0.0365   0.4253
beta on SMB         0.1870   0.0149
beta on HML        -0.0103   0.8794


In [13]:
reg_out.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.027
Date:                Tue, 17 Dec 2019   Prob (F-statistic):              0.108
Time:                        00:09:01   Log-Likelihood:                 3013.8
No. Observations:                1006   AIC:                            -6020.
Df Residuals:                    1002   BIC:                            -6000.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0008      0.000      2.131      0.033    6.46e-05       0.002
x1            -0.0365      0.046     -0.798      0.425      -0.126       0.053
x2             0.1870      0.077      2.439      0.015       0.037       0.337
x3            -0.0103      0.068     -0.152      0.879      -0.144       0.123
==============================================================================
Omnibus:                      219.904   Durbin-Watson:                   1.920
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1675.666
Skew:                          -0.783   Prob(JB):                         0.00
Kurtosis:                       9.126   Cond. No.                         203.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<b>(4 points) Question 3.</b> Use the daily returns for your other stock from Project 3 Part 1 to estimate an extended version of the Fama-French three-factor model that allows you to test whether the alpha on this stock is statistically significantly higher or lower during the last two years of the sample period than during the first two years. To implement this test, you must add an additional independent variable that takes the value of zero during the first half of your data (201510 to 201709) and takes the value of one during the second half of your data (201710 to 201909). Write an if-else statement that prints the p-value associated with this hypothesis test and states whether the coefficient is statistically significant at the 10-percent level or below.

In [14]:
ea = np.load('ea.npy')
dates_ea = np.load('dates_ea.npy')

In [15]:
print(ff_date.shape==dates_ea.shape)
print(ff_date[0]==dates_ea[0])
print(ff_date[-1]==dates_ea[-1])

True
True
True


In [16]:
# You want to distinguish dates through 20170930 from those after 20170930
listdate = ff_date.tolist()
print(listdate[0], listdate[-1])

20151001 20190930


In [17]:
halftime = []
for i in range(len(listdate)):
    if int(listdate[i]) >= 20171000:
        halftime.append(1)
    else:
        halftime.append(0)

interval = np.array(halftime).reshape(1006,1)

X = np.hstack((mktrf,smb,hml,interval))

dep_var1 = ea.reshape(1006,1) - rf

reg_out = reg_m(dep_var,X)

print('The p-value associated with the new interval variable is:{}'.format(float_formatter(reg_out.pvalues[-1])))

if (reg_out.params[-1]) >= 0.1:
    print('Therefore, the coefficent associated with this variable is not statistically significant,')
    if reg_out.params[0]>0:
        print('and the alpha of this stock is not significantly higher during the second half of the sample period.')
    else:
        print('and the alpha of this stock is not significantly lower during the second half of the sample period.\n')
else:
    print('Therefore, the coefficent associated with this variable is statistically significant,')
    if reg_out.params[0]>0:
            print('and the alpha of this stock is significantly higher during the second half of the sample period.')
    else:
        print('and the alpha of this stock is significantly lower during the second half of the sample period.\n')

reg_out.summary()      
      

      

The p-value associated with the new interval variable is: 0.3398
Therefore, the coefficent associated with this variable is not statistically significant,
and the alpha of this stock is not significantly higher during the second half of the sample period.


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.748
Date:                Tue, 17 Dec 2019   Prob (F-statistic):              0.137
Time:                        00:09:18   Log-Likelihood:                 3014.3
No. Observations:                1006   AIC:                            -6019.
Df Residuals:                    1001   BIC:                            -5994.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.001      0.835      0.404      -0.001       0.002
x1            -0.0359      0.046     -0.784      0.433      -0.126       0.054
x2             0.1895      0.077      2.470      0.014       0.039       0.340
x3            -0.0075      0.068     -0.111      0.912      -0.141       0.126
x4             0.0007      0.001      0.955      0.340      -0.001       0.002
==============================================================================
Omnibus:                      222.311   Durbin-Watson:                   1.922
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1692.386
Skew:                          -0.794   Prob(JB):                         0.00
Kurtosis:                       9.152   Cond. No.                         232.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""